Informação sobre o dataset

Number |Attribute Information
--|--
1...50 | Average, standard deviation, min, max and median of the Attributes 51...60 for the source of the current blog post. With source we mean the blog on which the post appeared. For example, myblog.blog.org would be the source of the post myblog.blog.org/post_2010_09_10 
51| Total number of comments before basetime 
52| Number of comments in the last 24 hours before the basetime 
53| Let T1 denote the datetime 48 hours before basetime, Let T2 denote the datetime 24 hours before basetime. This attribute is the number of comments in the time period between T1 and T2 
54| Number of comments in the first 24 hours after the publication of the blog post, but before basetime 
55| The difference of Attribute 52 and Attribute 53 
56...60| The same features as the attributes 51...55, but features 56...60 refer to the number of links (trackbacks), while features 51...55 refer to the number of comments. 
61| The length of time between the publication of the blog post and basetime 
62| The length of the blog post 
63...262| The 200 bag of words features for 200 frequent words of the text of the blog post 
263...269| binary indicator features (0 or 1) for the weekday (Monday...Sunday) of the basetime 
270...276| binary indicator features (0 or 1) for the weekday (Monday...Sunday) of the date of publication of the blog post 
277| Number of parent pages: we consider a blog post P as a parent of blog post B, if B is a reply (trackback) to  blog post P. 
278...280| Minimum, maximum, average number of comments that the parents received 
281| The target: the number of comments in the next 24 hours (relative to basetime)





In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn import linear_model
import os
from sklearn.externals import joblib

In [2]:
data_train = pd.read_csv('BlogFeedback/blogData_train.csv', header=None)
data_train = data_train[data_train[280] > 0]

Retirando as colunas de bag of words

In [4]:
t = [n for n in range(0,64)] + [n for n in range(263,280)]
#x_train = np.array(data_train.iloc[:,0:279])
x_train = np.array(data_train.iloc[:,t])
y_train = np.array(data_train.iloc[:,[280]])

In [6]:
regr = LinearSVR(random_state=0)
regr.fit(x_train, y_train.ravel())

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=0, tol=0.0001, verbose=0)

In [8]:
clf = linear_model.SGDRegressor()
clf.fit(x_train, y_train.ravel())

/home/gustavo/.virtualenvs/ia/local/lib/python3.4/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

Lendo os arquivos de teste

In [9]:
files_test = os.listdir("BlogFeedback/")
files_test = [f for f in files_test if f.startswith("blogData_test")]
data_tests = [pd.read_csv(os.path.join("BlogFeedback", f), header=None) for f in files_test]
data_tests = pd.concat(data_tests)

In [10]:
#x_test = np.array(data_tests.iloc[:,0:279])
x_test = np.array(data_tests.iloc[:,t])
y_test = np.array(data_tests.iloc[:,[280]])
print(x_test.shape)
print(y_test.shape)

(7624, 81)
(7624, 1)


In [11]:
print(regr.score(x_test, y_test))

-1.62846310756


In [12]:
joblib.dump(regr, 'LinearSVR.sav') 

['LinearSVR.sav']

In [13]:
print(clf.score(x_test, y_test))

-9.47614552376e+29


In [14]:
joblib.dump(regr, 'SGDRegressor.sav') 

['SGDRegressor.sav']